# Prompt engineering for Claude 3 using API via claudetools library

The parameter explanation is provided below.

- model: The name of the model from the Claude 3 family.
- messages: Messages transferred between the user and the assistant.
- tools: Set of function specification to use.
- tool_choice: User a particular function. By default the value is None. The model will figure out the function to call and provide the related parameters. If a specific function needs to be called provide {"name": "function name"} in the tool_choice argument.
- multiple_tools: Defaults to False. Can be set to True to get multiple function calls.
- attach_system: Defaults to None. Can also accept string which will be attached as part of the system prompt.
- max_tokens: As mentioned above, no default value of max_tokens is assumed. Hence, please provide max_tokens to avoid getting an error.

In [1]:
# !pip install claudetools 
# # for interaction with claude3 model
# !pip install python-dotenv


  Obtaining dependency information for claudetools from https://files.pythonhosted.org/packages/79/91/2ddd25c6da3fb2f7c87e50574dbb8ff325c2ee490db7381635445aaa4c5a/claudetools-0.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx==0.25.0 from https://files.pythonhosted.org/packages/33/0d/d9ce469af019741c8999711d36b270ff992ceb1a0293f73f9f34fdf131e9/httpx-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for anthropic==0.19.1 from https://files.pythonhosted.org/packages/83/be/2127c0cc1aceb1109ae4522e92038626954d747da2b4f359538a3cb79018/anthropic-0.19.1-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic==2.4.2 from https://files.pythonhosted.org/packages/73/66/0a72c9fcde42e5650c8d8d5c5c1873b9a3893018020c77ca8eb62708b923/pydantic-2.4.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for httpcore<0.19.0,>=0.18.0 from https://fi

# Trying 1 Claude3 model and 1 prompt
- with Dependency prompt (system prompt)
    - claude-3-opus
    - "spawn 6 trees in a grid"


In [2]:
import asyncio
# from claudetools.tools.tool import Tool
# from pydantic import BaseModel, Field
from typing import List, Dict
import json
import requests
import pandas as pd
import os

from config import ANTHROPIC_API_KEY
import anthropic

True

In [8]:
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

In [1]:
prompt = """
    spawn 6 trees in a grid
"""

# dependency prompt to attach to the main system prompt
# DEPENDENCY_PROMPT = """You are a helpful assistant that helps a user with their tasks and todos. The user can add a todos, mark todos as completed, or reopen certain todos.
# The user can provide multiple actions at once so you've to break those down and call the appropriate functions in the correct sequence."""
DEPENDENCY_PROMPT = """
I need your help to understand and generate some commands in the form of Shards scripts to be sent to our game creation system (Formabble), which is Right Handed with Y as up axis, from whatever the user inputs.

### A bit about Shards:

String: "hello world"
Integer: 1
Float: 1.0
Float2: @f2(1.0 2.0)
Float3: @f3(1.1 2.0 3.0)
We also have @f4
Int vectors also: @i2, @i3, @i4, @i8 and @i16@ is a prefix for our built-ins, definitions and templates to distinguish between variables
; is the line comment symbol, ignore comments

Shards ["Hello " @name] | String.Join | Log >= message [] is a sequence, in the above you can see that @name is something defined, while message is a variable we assign with >= which is a mutable assignment operator, as you can see in Shards everything flows from left to right from top to bottom, even assignments.
> is used to update mutable values
= is used to assign am immutable reference

Shards Count(seq) When({IsMore(0) | And | IsMore(selected-idx)} {   ; DO SOMETHING With Count(seq) output }) 
In this Shards script, we start by counting the elements in seq. Then, we evaluate two conditions in sequence: first, we check if the count is greater than zero.
If this is true, we then check if the count is also greater than selected-idx. Both conditions must be satisfied for the script to execute the code within the block

Types are strict, so when you use or pass a constant 0 will be an integer 0.0 will be float
Same when declaring variables:
0.0 >= float-var0 >= int-var
Types are also strict when doing math! so please use right types, you can use ToFloat, ToInt etc to convert.
Variables must be unique! Don't Set with same name. Unless it's an Update (>)
Example, idx | Math.Mod(10), if you need float output use idx | Math.Mod(10) | ToFloat
You often mistake even the opposite, if for some reason idx is a float you also need to do this: idx | ToInt | Math.Mod(10) | ToFloat
E.g.
idx | ToFloat | Math.Divide(5.0) | Math.Floor = row row | Math.Mod(2) | ToFloat | Math.Multiply(1.0) = zExtraOffset ; <------- WRONG, should be: row | ToInt| Math.Mod(2) | ToFloat | Math.Multiply(1.0) = zExtraOffset
Or
idx | Math.Mod(5) = column column | ToFloat | Math.Multiply(1.75) = zOffset ; column came from Mod output, so was a Int!

Math is done like this:
time | Math.Multiply(0.2) | Math.AxisAngleX | Math.Rotation ; Rotation X | Math.MatMul((time | Math.Multiply(0.7) | Math.AxisAngleY | Math.Rotation)) ; Rotation Y | Math.MatMul((time | Math.Multiply(0.9) | Math.AxisAngleZ | Math.Rotation)) ; Rotation Z | Log+ is Math.Add, - is Math.Subtract, * is Math.Multiply, / is Math.Divide
We don't use regular operators such as (+,-,*,/) in shards and math is always from left to right we don't have math expressions like C,
so this is wrong @f3(1 + index1 2 1 + index2), but it can be achieved using () construct which will wrap the operation and create a temp variable such as:
@f3((1 | Math.Add(x)) 2 (1 | Math.Add(y)))

Also keep in mind that Math.Floor will output a float! so you might need to ToInt its output!!

Also Shards is sort of strongly typed so if you wanna do something like this:
f3(0.0 (idx | Math.Multiply(1.0)) 0.0)
it should be:
@f3(0.0 (idx | ToFloat | Math.Multiply(1.0)) 0.0)

Loops:
0 >= idx ; we need to manage manually the index Repeat({   Msg("Hello")   Math.Inc(idx) } Times: 3)
or
ForRange(0 2 {   = idx ; we get automatically the current index   Msg("Hello") })

Seq access and creation
1 >> s 2 >> s  s | Take(0) | Assert.Is(1)

Tables access and creation
{a: 1 b: 2} = t ; actually any JSON will do, our tables are like json tables  t | Take("a") | Assert.Is(1)
Always prefer declaring tables in JSON style though! and Immutable.

Random numbers!
RandomFloat(Max: 1.0) | IsLess(1.0) | Assert.Is(true) RandomInt(Max: 100) | IsLess(100) | Assert.Is(true)

Remember:
this is wrong:
@f3(randomX 0.0 y | Math.Multiply(10.0)) = position
it should be:
@f3(randomX 0.0 (y | Math.Multiply(10.0))) = position as you need () for such lambda behavior

Some Formabble specific functionality:
* Formalize
* Description
* Spawn or create an entity (a form in our case) from a form asset in the environment or scene (called domain in our case).
* Example
* {Graphics: {"fbl/pose": {translation: @f3(0.0 0.0 0.0) rotation: @f4(0.0 0.0 0.0 1.0) scale: @f3(1.0 1.0 1.0)}}} | Fbl.Formalize("__ASSET_ID__")
* Notes
* Important: Always include all TRS components, even if default!!
* Important: Rotation is always a quaternion
* Dispatch
* Description
* Dispatches a variable change to the requested form in the right subsystem
* Examples
* Math.Decompose = gizmo-trs | gizmo-trs | Take("translation") | Fbl.Dispatch(form-id WirePoolType::Graphics "fbl/pose" Key: "translation") (Key is optional)
* 10.0 | Fbl.Dispatch("cube1" WirePoolType::Graphics "speed")

Now from now on you will wait for user prompts and generate commands.
For now just focus on Formalizing forms. User input will be natural language instructions.
Please output a Shard script with all the code to achieve the requested instructions.
Be brief, omit validation for now.
Oh and please from now on just answer queries related to Formabble and Shards.

Examples:
A RANDOM FOREST
0 >= idx ForRange(0 99 {   idx | ToFloat | Math.Divide(10.0) | Math.Floor = x   idx | Math.Mod(10) = z   RandomFloat(Max: 1.0) | Math.Multiply(10.0) = randomX   RandomFloat(Max: 1.0) | Math.Multiply(10.0) = randomZ   @f3((x | Math.Add(randomX)) 0.0 (z | ToFloat | Math.Add(randomZ))) = position   {Graphics: {"fbl/pose": {translation: position rotation: @f4(0.0 0.0 0.0 1.0) scale: @f3(1.0)}}} | Fbl.Formalize("d-treeDefault")   Math.Inc(idx) })

A GRID OF BLOCKS
0 >= idx ForRange(0 99 {   idx | ToFloat | Math.Divide(10.0) | Math.Floor = x   idx | Math.Mod(10) | ToFloat = z   @f3(x 0.0 z) = position   {Graphics: {"fbl/pose": {translation: position rotation: @f4(0.0 0.0 0.0 1.0) scale: @f3(1.0)}}} | Fbl.Formalize("d-block")   Math.Inc(idx) })

ALWAYS PREFER SIMPLE CODE, DON'T NEED TO OPTIMIZE, UNROLL OPERATIONS ETC!
Your reply should look like this:
MARKDOWN SHARDS CODE BLOCK TO EVAL
LINE BREAK
COMMENTS
"""
# set up the user messages
user_messages = [{
    "role":
    "user",
    "content":
    DEPENDENCY_PROMPT + "\n" + prompt}]   


In [22]:

full_prompt = DEPENDENCY_PROMPT + "\n" + prompt


client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=ANTHROPIC_API_KEY,
)

message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    system="You are a Shards programming expert.",
    messages=[
        {"role": "user", "content": full_prompt}
    ]
)

print(message.content)

2024-04-09 16:10:52 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


[ContentBlock(text='```\n0 >= idx\nForRange(0 5 {\n  idx | ToFloat | Math.Divide(2.0) | Math.Floor = x\n  idx | Math.Mod(2) | ToFloat = z\n  @f3((x | Math.Multiply(5.0)) 0.0 (z | Math.Multiply(5.0))) = position\n  {Graphics: {"fbl/pose": {translation: position rotation: @f4(0.0 0.0 0.0 1.0) scale: @f3(1.0 1.0 1.0)}}} | Fbl.Formalize("d-treeDefault") \n  Math.Inc(idx)\n})\n```\n\nThis will spawn 6 trees in a 2x3 grid pattern. \n- It loops 6 times using ForRange\n- Calculates the x and z positions by dividing and modding the index by 2 to get coordinates in a grid\n- Multiplies x and z by 5 to space out the trees\n- Sets the position using the calculated x, 0 for y, and z \n- Formalizes a "d-treeDefault" asset at each position\n\nLet me know if you have any other questions!', type='text')]


In [23]:
print(message.content[0].text)

```
0 >= idx
ForRange(0 5 {
  idx | ToFloat | Math.Divide(2.0) | Math.Floor = x
  idx | Math.Mod(2) | ToFloat = z
  @f3((x | Math.Multiply(5.0)) 0.0 (z | Math.Multiply(5.0))) = position
  {Graphics: {"fbl/pose": {translation: position rotation: @f4(0.0 0.0 0.0 1.0) scale: @f3(1.0 1.0 1.0)}}} | Fbl.Formalize("d-treeDefault") 
  Math.Inc(idx)
})
```

This will spawn 6 trees in a 2x3 grid pattern. 
- It loops 6 times using ForRange
- Calculates the x and z positions by dividing and modding the index by 2 to get coordinates in a grid
- Multiplies x and z by 5 to space out the trees
- Sets the position using the calculated x, 0 for y, and z 
- Formalizes a "d-treeDefault" asset at each position

Let me know if you have any other questions!


# Trying 3 Claude3 models and 5 prompts

In [25]:
models = ["claude-3-opus-20240229", "claude-3-sonnet-20240229","claude-3-haiku-20240307"]
prompts = [DEPENDENCY_PROMPT + "spawn 6 trees in a grid", 
           "Generate 6 trees spread randomly on a 3x3 grid using d-TreePine, starting from x and z position of 1.0 Keep the y-axis at 0.0",
           "Generate 10 flowers spread randomly on a 2x2 grid using d-FlowerRed, starting from x and z position of 1.0 Keep the y-axis at 0.0 ",
           "Generate a 6x6 hexagonal grid using d-BlockHexagon tiles. Keep the y-axis at -1.0 Offset tile’s x position by 0.5 and z position by 0.865 so they form a cohesive hexagonal grid",
           "Starting from x position 4 and z position 4, generate a 4x4 hexagonal grid using d-BlockSnowHexagon tiles. Keep the y-axis at -1.0 Offset tile’s x position by 0.5 and z position by 0.865 so they form a cohesive hexagonal grid."
           ] # 5 prompts

In [29]:
# Create empty DataFrame
df = pd.DataFrame(columns=['Prompt', 'Model', 'Completion'])

for model in models:
    for prompt in prompts:
        # call the tool with the required parameters
        message = client.messages.create(
            model=model,
            max_tokens=1000,
            temperature=0.0,
            system="You are a Shards programming expert.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        # Appending the result to the DataFrame
        df = pd.concat([df, pd.DataFrame({'Prompt': prompt, 'Model': model, 'Completion': [message.content[0].text]})], ignore_index=True)

# Saving the results to a CSV file
df.to_csv("claude_model_completions.csv", index=False)

print("Completions saved to model_completions.csv")


2024-04-09 16:19:30 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:20:02 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:20:37 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:21:01 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:21:26 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:21:32 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:21:45 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:21:56 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-04-09 16:22:09 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 

Completions saved to model_completions.csv
